# LanternDB
In this notebook, we will demonstrate how to use LanternDB, for accessing and querying data efficiently. LanternDB is designed to work seamlessly with modern analytical workloads, making it a powerful tool for data analysis, research, and question-answering systems.

To begin, ensure you have LanternDB installed in your Python environment. You can easily install it using `pip install pgcopg2`. so you can start querying data directly in your local environment without any additional setup.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/osllmai/inDox/blob/master/cookbook/indoxArcg/lanterndb.ipynb)

In [ ]:
!pip install indoxArcg
!pip install psycopg2
!pip install semantic_text_splitter
!pip install sentence-transformers

## Setting Up the Python Environment

If you are running this project in your local IDE, please create a Python environment to ensure all dependencies are correctly managed. You can follow the steps below to set up a virtual environment named `indoxArcg`:

### Windows

1. **Create the virtual environment:**
```bash
python -m venv indoxArcg
```
2. **Activate the virtual environment:**
```bash
indoxArcg\Scripts\activate
```

### macOS/Linux

1. **Create the virtual environment:**
   ```bash
   python3 -m venv indoxArcg
```

2. **Activate the virtual environment:**
    ```bash
   source indoxArcg/bin/activate
```
### Install Dependencies

Once the virtual environment is activated, install the required dependencies by running:

```bash
pip install -r requirements.txt
```


### Load Hugging face API key 


In [1]:
import os
from dotenv import load_dotenv

load_dotenv('api.env')
HUGGINGFACE_API_KEY = os.getenv('HUGGINGFACE_API_KEY')

Initialize a language model and an embedding model using the indox library with Hugging Face . The HuggingFaceModel class is used to create an instance of the Mistral-7B-Instruct model for tasks like question answering

In [ ]:
from indoxArcg.llms import HuggingFaceModel
from indoxArcg.embeddings import HuggingFaceEmbedding
mistral_qa = HuggingFaceModel(api_key=HUGGINGFACE_API_KEY,model="mistralai/Mistral-7B-Instruct-v0.2")
embed = HuggingFaceEmbedding(api_key=HUGGINGFACE_API_KEY,model="multi-qa-mpnet-base-cos-v1")

INFO: Initializing HuggingFaceModel with model: mistralai/Mistral-7B-Instruct-v0.2
INFO: HuggingFaceModel initialized successfully
INFO: Initialized HuggingFaceEmbedding with model: multi-qa-mpnet-base-cos-v1


### Load sample text

In [ ]:
!wget https://raw.githubusercontent.com/osllmai/inDox/master/Demo/sample.txt


In [4]:
file_path = 'sample.txt'

In [5]:
with open(file_path, "r") as file:
    text = file.read()

use the `SemanticTextSplitter` class from the indox library to divide a large text into smaller, manageable chunks

In [ ]:
from indoxArcg.splitter import SemanticTextSplitter
splitter = SemanticTextSplitter(400)
content_chunks = splitter.split_text(text)

### Set up vector store
Set up a vector store using the `LanternDB` class from the indox library.

In [ ]:
from indoxArcg.vector_stores import LanternDB
connection_params = {
    "dbname": "DatabaseName",
    "user": "User",
    "password": "password",
    "host": "host",
    "port": "port"
}
lantern_db = LanternDB(
    collection_name="Vector_collection",
    embedding_function=embed,
    connection_params=connection_params,
    dimension=768,
)

Connected to LanternDB collection 'Vector_collection'
Collection 'Vector_collection' created successfully.


### Storing Data in the Vector Store


In [8]:
lantern_db.add_texts(content_chunks)

INFO: Embedding documents
INFO: Starting to fetch embeddings for texts using model: SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)


Batches: 100%|██████████| 1/1 [00:00<00:00,  2.40it/s]


Inserted 9 documents into collection 'Vector_collection'.


['852d6289-a105-48f3-94e8-4169f109abb7',
 '9de00e1c-736f-41f7-a4cf-2337bb62d5d7',
 'f00c66ba-ba5e-4928-b7bc-980688a7efd2',
 '21c94f7b-ee16-4c49-b914-da0c2e5247c4',
 '09323629-9f81-4236-9619-c84e025f3b8f',
 'b88cdb3a-79dd-48bf-b3d3-f44d7ee258be',
 '126dd7fb-4dba-4189-bfaa-c220fd69570e',
 'c223a9a1-232e-493b-aa90-129a21d05d19',
 'ebbcb9cc-c00e-4036-a942-28fb1f079d6e']

In [9]:
query = "How cinderella reach her happy ending?"


In [10]:
from indoxArcg.pipelines.rag import RAG
retriever = RAG(llm=mistral_qa,vector_store=lantern_db,enable_web_fallback=False,top_k= 5)
answer = retriever.infer(query)


INFO: Retrieving context and scores from the vector database
INFO: Embedding documents
INFO: Starting to fetch embeddings for texts using model: SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)


Batches: 100%|██████████| 1/1 [00:00<00:00, 54.19it/s]


INFO: Generating answer without document relevancy filter
INFO: Answering question
INFO: Sending request to Hugging Face API
INFO: Received successful response from Hugging Face API
INFO: Query answered successfully


In [11]:
answer

"Cinderella reached her happy ending by attending the royal ball in a beautiful dress and golden slippers that were magically provided to her by a little white bird that lived on a tree she had once wept and prayed under. The king's son fell in love with her and was determined to marry her, so he searched for the woman whose foot the golden slipper fit. When the two stepsisters attempted to wear the slipper, it didn't fit because Cinders"